In [1]:
import pandas as pd
import numpy as np

# 3 parts:
# kostenschätzung
# ausgabenberechnung (bilanz berechnen => schätzung vs. tatsächliche kosten)
# kostenvergleich

In [2]:
path_gewerke = r'C:\Users\Michael\Google Drive\Projekt_Palast\Finanzen\Kostenberechung\Kostenberechnung_Gewerke.xlsx'
df_gewerke = pd.read_excel(path_gewerke, skiprows=1, converters={'ID':str})
# bring all position descriptions to column 'Beschreibung'
df_gewerke.loc[~df_gewerke['Unnamed: 3'].isnull(), 'Beschreibung'] = df_gewerke.loc[~df_gewerke['Unnamed: 3'].isnull()]['Unnamed: 3']
df_gewerke.loc[~df_gewerke['Unnamed: 4'].isnull(), 'Beschreibung'] = df_gewerke.loc[~df_gewerke['Unnamed: 4'].isnull()]['Unnamed: 4']
# remove all columns except ID and Beschreibung
df_gewerke = df_gewerke[['ID', 'Beschreibung']]


# TODO plausibility checks ID == np.nan
# TODO plausibility check ascending ids!
# TODO check if all ids < level remain constant inside the group ( for example 10.10, 10.20 10.30 ok, 10.10, 20.10, 10.20 wrong!)
# maybe this makes no sense!..

In [3]:
def get_df_with_extended_id_cols(df, level=None):
    df['ID'] = df['ID'].astype(str)
    df['level'] = df['ID'].str.count('\\.')
    df['id_list'] = df['ID'].str.split('\\.')
    max_level = df['level'].max()
    print(max_level)
    for cur_level in range(max_level+1):
        df[f'id_{cur_level}'] = np.nan
        mask_level = df['level'] >= cur_level
        df.loc[mask_level, f'id_{cur_level}'] = df.loc[mask_level]['id_list'].apply(lambda x: x[cur_level])
    return df

In [4]:
df_gewerke = get_df_with_extended_id_cols(df_gewerke)


4


In [5]:
# calculation total cost
dir_angebote = r'C:\Users\Michael\Google Drive\Projekt_Palast\Finanzen\Kostenberechung\Analyse'
# TODO this will be manually chooseable in the dashapp
sheet_nr = 1
import os
import glob
files = os.listdir(dir_angebote)
print(files)

list_df_offers = []
for file_path in files:
    if not file_path.endswith('.xlsx') or file_path.startswith('~'):
        continue
    path_angebot = os.path.join(dir_angebote, file_path)
    df_angebot = pd.read_excel(path_angebot, skiprows=3, converters={'ID':str}, sheet_name=sheet_nr)
    df_angebot = df_angebot[df_angebot.columns.drop(list(df_angebot.filter(regex='Unnamed')))]
    df_angebot = df_angebot.loc[~df_angebot['ID'].isnull()]
    df_angebot.loc[:, 'ID'] = df_angebot['ID'].astype(str)
    # add brutto column
    if "Brutto" not in df_angebot.columns:
        df_angebot["Brutto"] = "Nein"
    # add UST everywhere Brutto == Nein
    df_angebot["price_brutto"] = df_angebot["Gesamtpreis"]
    df_angebot["factor"] = 1.0
    df_angebot.loc[df_angebot["Brutto"] != "Ja", "factor"] = 1.2
    df_angebot.loc[:, "price_brutto"] =  df_angebot["price_brutto"] * df_angebot["factor"]
    
    # TODO add source sheet as well
    df_angebot.loc[:, 'file_source'] = file_path
    list_df_offers.append(df_angebot)

df_offers = pd.concat(list_df_offers)
max_level = df_gewerke['level'].max()
df_offers = get_df_with_extended_id_cols(df_offers, max_level)

#TODO check if preisangaben für subposition + position gemacht wurde => das unterstützen wir nicht zurzeit (könnte man aber ändern)
# TODO combine with estimates (schätzungen excel files, immer nur schätzung verwenden wenn kein angebot vorhanden)
# 
df_offers.groupby(['id_0']).sum()

['Baumeister.xlsx', 'Bautischler.xlsx', 'Bodenleger.xlsx', 'Dachdecker.xlsx', 'desktop.ini', 'Fenster.xlsx', 'Fliesenleger.xlsx', 'Installateur.xlsx', 'Klaeranlage.xlsx', 'Kostenschaetzung.xlsx', 'Nebenkosten.xlsx', 'Zimmerer.xlsx', '~$Zimmerer.xlsx']


C:\Users\Michael\Projekte\HausbauDev\building_cost_analysis\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


4


,Menge,E-Preis,Gesamtpreis,price_brutto,factor,level
id_0,,,,,,
10,1.00,0.0000,0.00,0.000,1.0,0
20,5407.12,349124.0017,446427.04,516251.800,163.2,438
30,55.00,46743.7900,50347.79,58657.348,16.2,28
40,11.00,29225.0000,29225.00,29225.000,11.0,11


In [6]:
df_gewerke.to_csv('./data/temp.csv')

In [7]:
df_offers.to_csv('./data/temp_angebot.csv')

In [8]:
# TODO determine duplicated positions
duplicated_id = df_offers[df_offers[['ID']].duplicated()]['ID'].values
print('Duplikate:')
df_offers.loc[df_offers['ID'].isin(duplicated_id)]

Duplikate:


,ID,Beschreibung,Menge,Einheit,Montage Bauseits,E-Preis,Gesamtpreis,Brutto,price_brutto,factor,file_source,level,id_list,id_0,id_1,id_2,id_3,id_4


In [9]:
def append_rolled_up_sum_to_df(df_combined):
    max_level = df_combined['level'].max()
    list_level_sums = []
    for cur_level in range(max_level+1):
        list_ids = []
        for lev in range(cur_level + 1):
            list_ids.append(f'id_{lev}')
        df_level_count = df_combined.groupby(list_ids).count()
        df_level_sum = df_combined.groupby(list_ids).sum()
        df_level_sum['summed_up'] = df_level_sum['price_brutto']
        df_level_count['prices_existing'] = False
        df_level_count.loc[df_level_count['price_brutto'] != 0, 'prices_existing'] = True
        df_level_sum = df_level_sum[['summed_up']].combine_first(df_level_count[['prices_existing']])
        df_level_sum['level'] = cur_level
        df_level_sum = df_level_sum.reset_index()
        df_level_sum['ID'] = df_level_sum['id_0']
        for lev in range(1, cur_level + 1):
            df_level_sum['ID'] += "." + df_level_sum[f'id_{lev}']

        list_level_sums.append(df_level_sum)
    df_rolled_up_sum = pd.DataFrame()

    for df_sum in list_level_sums:
        df_rolled_up_sum = df_rolled_up_sum.combine_first(df_sum.set_index('ID'))
    df_ret = df_combined.combine_first(df_rolled_up_sum[['prices_existing', 'summed_up']])
    df_ret.to_csv('./data/summed_up_test.csv')
    return df_ret

df_combined = df_offers.set_index('ID').combine_first(df_gewerke.set_index('ID'))
df_combined.to_csv('./data/combined_test.csv')

# TODO calculate prices for each level
df_combined = append_rolled_up_sum_to_df(df_combined)




In [10]:
max_level = df_combined['level'].max()
list_ids = []
for lev in range(max_level + 1):
    list_ids.append(f'id_{lev}')
    df_combined[f'id_{lev}'] = df_combined[f'id_{lev}'].astype(float).fillna(0)
df_sorted = df_combined.sort_values(list_ids)
df_combined.sort_values(list_ids).to_csv('./data/sorted.csv')

In [11]:
df_sorted.loc[df_sorted['level'] == 0]

,Beschreibung,Brutto,E-Preis,Einheit,Gesamtpreis,Menge,Montage Bauseits,factor,file_source,id_0,id_1,id_2,id_3,id_4,id_list,level,price_brutto,prices_existing,summed_up
ID,,,,,,,,,,,,,,,,,,,
10,Aufschließung und Baureifmachung,Ja,0.0,PA,0.0,1.0,Nein,1.0,Kostenschaetzung.xlsx,10.0,0.0,0.0,0.0,0.0,[10],0,0.0,True,0.000
20,Bauwerkskosten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,0.0,0.0,0.0,0.0,[20],0,NaN,True,516251.800
30,Aussenanlagen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,0.0,0.0,0.0,0.0,[30],0,NaN,True,58657.348
40,Nebenkosten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,0.0,0.0,0.0,0.0,[40],0,NaN,True,29225.000


In [12]:
# TODO determine missing positions
def get_missing_positions(df_sorted):
    max_level = df_sorted['level'].max()
    list_ids = []
    df_missing = df_sorted.copy()
    df_missing['display_missing'] = False
    df_missing['upper_level_missing'] = False
    df_missing['upper_level_inserted'] = False
    for lev in range(max_level + 1):
        mask_cur_level = (df_missing['level'] == lev) & (df_missing['prices_existing'] == False) & (df_missing['upper_level_missing'] == False)
        df_missing.loc[mask_cur_level, 'display_missing'] = True
        # set upper level missing
        for row_id, missing_row in df_missing[mask_cur_level].iterrows():
            mask = (df_missing['level'] > lev)
            for sublev in range(lev+1):
                mask = mask & (df_missing[f'id_{sublev}'] == missing_row[f'id_{sublev}'])
            df_missing.loc[mask, 'upper_level_missing'] = True
        # set upper level inserted
        mask_inserted = (df_missing['level'] == lev) & (~df_missing['price_brutto'].isnull())
        for row_id, missing_row in df_missing[mask_inserted].iterrows():
            mask = (df_missing['level'] > lev)
            for sublev in range(lev+1):
                mask = mask & (df_missing[f'id_{sublev}'] == missing_row[f'id_{sublev}'])
            df_missing.loc[mask, 'upper_level_inserted'] = True
    return df_missing.loc[(df_missing['display_missing'] == True) & ((df_missing['upper_level_inserted'] == False))]

In [13]:
get_missing_positions(df_sorted).to_csv('./data/missing.csv')
get_missing_positions(df_sorted)[['Beschreibung']]

,Beschreibung
ID,
20.30.40,Beschattungsschacht Neubau für Rafstore
20.30.50.30,Sonstige Stützen Zubau
20.40.10.20,Facharbeiter
20.40.10.30,Sonstige Kosten
20.40.20.20,Dachkonstruktion Neu Bestandsdach (wschlich ni...
20.40.20.30.70,Bitumenlage ZIB15 Flämmschutz und Noteindeckung
20.40.20.30.80,Tropfblech mit Insektenschutzgitter im Traufen...
20.50.40,Unterkonstruktion Decke Anbau (Freisitz)
20.70.60,Technikraum Boden Abdichtung


In [14]:
# TODO Gesamtpreis Brutto column einfügen
df_sorted.groupby(['id_0']).sum()

,E-Preis,Gesamtpreis,Menge,factor,id_1,id_2,id_3,id_4,level,price_brutto,prices_existing,summed_up
id_0,,,,,,,,,,,,
10.0,0.0000,0.00,1.00,1.0,160.0,170.0,0.0,0.0,17,0.000,1,0.000
20.0,349124.0017,446427.04,5407.12,163.2,25510.0,11300.0,6650.0,1930.0,854,516251.800,190,1707281.688
30.0,46743.7900,50347.79,55.00,16.2,1230.0,690.0,60.0,0.0,63,58657.348,19,178776.836
40.0,29225.0000,29225.00,11.00,11.0,780.0,0.0,0.0,0.0,12,29225.000,12,58450.000


In [15]:
df_sorted.loc[df_sorted['level'] == 0][['summed_up']].sum() * 1.0# 5 % sicherheitsreserve

summed_up    604134.148
dtype: float64

In [16]:
df_sorted.loc[df_sorted['level'] == 0]

,Beschreibung,Brutto,E-Preis,Einheit,Gesamtpreis,Menge,Montage Bauseits,factor,file_source,id_0,id_1,id_2,id_3,id_4,id_list,level,price_brutto,prices_existing,summed_up
ID,,,,,,,,,,,,,,,,,,,
10,Aufschließung und Baureifmachung,Ja,0.0,PA,0.0,1.0,Nein,1.0,Kostenschaetzung.xlsx,10.0,0.0,0.0,0.0,0.0,[10],0,0.0,True,0.000
20,Bauwerkskosten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,0.0,0.0,0.0,0.0,[20],0,NaN,True,516251.800
30,Aussenanlagen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,0.0,0.0,0.0,0.0,[30],0,NaN,True,58657.348
40,Nebenkosten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,0.0,0.0,0.0,0.0,[40],0,NaN,True,29225.000


In [17]:
df_sorted.loc[df_sorted['level'] == 1]

,Beschreibung,Brutto,E-Preis,Einheit,Gesamtpreis,Menge,Montage Bauseits,factor,file_source,id_0,id_1,id_2,id_3,id_4,id_list,level,price_brutto,prices_existing,summed_up
ID,,,,,,,,,,,,,,,,,,,
10.10,Erschließung Medienversorgung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,0.0,0.0,0.0,"[10, 10]",1,NaN,False,0.000
10.20,Verkehrserschließung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,20.0,0.0,0.0,0.0,"[10, 20]",1,NaN,False,0.000
10.30,Baureifmachung / Abbruch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,30.0,0.0,0.0,0.0,"[10, 30]",1,NaN,False,0.000
20.20,Baumeisterarbeiten + Bodenaufbau OG (aus Koste...,Nein,108337.0,PA,108337.0,1.0,Nein,1.2,Baumeister.xlsx,20.0,20.0,0.0,0.0,0.0,"[20, 20]",1,130004.4,True,130004.400
20.30,Holzbau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,30.0,0.0,0.0,0.0,"[20, 30]",1,NaN,True,44142.960
20.40,Zimmermannsarbeiten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,40.0,0.0,0.0,0.0,"[20, 40]",1,NaN,True,57809.976
20.50,Trockenbauarbeiten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,50.0,0.0,0.0,0.0,"[20, 50]",1,NaN,True,23298.084
20.60,Vollwärmeschutz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,60.0,0.0,0.0,0.0,"[20, 60]",1,NaN,True,17933.400
20.70,"Dachdecker, Spengler- und Schwarzdecker",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,70.0,0.0,0.0,0.0,"[20, 70]",1,NaN,True,39699.924
